# Use OpenRouter With OpenAI Agents SDK

Setup Prerequisite:

1. [Signup at OpenRouter](https://openrouter.ai/)
2. [Create an API Key](https://openrouter.ai/settings/keys)
2. Select a Free Model (you can continue as we are using a free model here)

## Free and Paid Models

The OpenRouter supports the latest DeepSeek V3 0324 and 50+ other models for free. Most of them support the defacto standard: OpenAI Chat Completion API.


If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

**See all Models List: https://openrouter.ai/models**

Note: OpenRouter do not charge anything extra at inference time.

## Rate Limiting and Crediting

There are a few rate limits that apply to certain types of requests, regardless of account status:

- Free limit: If you are using a free model variant (with an ID ending in :free), then you will be limited to 20 requests per minute and 200 requests per day.

If your account has a negative credit balance, you may see 402 errors, including for free models. Adding credits to put your balance above zero allows you to use those models again.

[Reference](https://openrouter.ai/docs/api-reference/limits)

## Install OpenAI Agents Dep.

In [8]:
!pip install -Uq openai-agents

In [9]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [10]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [11]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "mistralai/mistral-small-3.1-24b-instruct:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [12]:
import requests
import json

response = requests.post(
  url=f"{BASE_URL}/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1745066182-dl2vL9l7FA0izDwF2dBm', 'provider': 'Chutes', 'model': 'mistralai/mistral-small-3.1-24b-instruct', 'object': 'chat.completion', 'created': 1745066182, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "The meaning of life is a philosophical question that has been debated for centuries, and it doesn't have one definitive answer as it can vary greatly depending on personal beliefs, religious or spiritual views, and philosophical persuasions. Here are a few perspectives:\n\n1. **Existentialism**: Existentialists like Jean-Paul Sartre argued that life has no inherent meaning, and it's up to each individual to create their own purpose.\n\n2. **Religious and Spiritual Views**: Many religions provide their own answers. For example:\n   - In Christianity, the purpose of life might be seen as loving and serving God.\n   - In Buddhism, the purpose could be achieving enlightenment.\

In [13]:
data = response.json()
data['choices'][0]['message']['content']

"The meaning of life is a philosophical question that has been debated for centuries, and it doesn't have one definitive answer as it can vary greatly depending on personal beliefs, religious or spiritual views, and philosophical persuasions. Here are a few perspectives:\n\n1. **Existentialism**: Existentialists like Jean-Paul Sartre argued that life has no inherent meaning, and it's up to each individual to create their own purpose.\n\n2. **Religious and Spiritual Views**: Many religions provide their own answers. For example:\n   - In Christianity, the purpose of life might be seen as loving and serving God.\n   - In Buddhism, the purpose could be achieving enlightenment.\n   - In Hinduism, the purpose might be fulfilling one's dharma and ultimately, Moksha (liberation from the cycle of life and death).\n\n3. **Hedonism**: Hedonists believe the purpose of life is to seek pleasure and happiness.\n\n4. **Altruism**: Some people find meaning in life through helping others and making the

## 2. Using OpenAI Agents SDK

In [14]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)

set_tracing_disabled(disabled=True)

async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent,
        "Tell me about recursion in programming.",
    )
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

A function calls
Itself, like a serpent's tail
looping, it begins


# OpenRouter 404 Error Solution

## Error - No endpoints found matching
```python
NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy. Enable prompt training here: https://openrouter.ai/settings/privacy', 'code': 404}}
```

## Cause
This error occurs when OpenRouter API can't find endpoints matching your data policy, typically because prompt training is disabled.

## Solution

1. **Enable Prompt Training**:
   - Visit [OpenRouter Privacy Settings](https://openrouter.ai/settings/privacy)
   - Toggle ON "Prompt Training" option

2. **Re-run your code** after enabling

![OpenRouter Settings Screenshot](https://github.com/panaversity/learn-agentic-ai/blob/main/01_ai_agents_first/02_openrouter/openrouter.png?raw=1)
*(Example: Enable prompt training in privacy settings)*

## Prevention
Keep prompt training enabled for uninterrupted API access.
```